# Simple Event Log Viewer

This note book provides a user with a method of changing the output of User Activity Monitor to allow the user to drop an entire column of output or change the contents of that column in a way to preserve the anonymity of the column but maintain consistency for reporting.

For the full list of columns, please refer to: 
https://docs.datarobot.com/en/docs/api/reference/public-api/analytics.html#get-apiv2eventlogs

## Required Permissions

The user that executes this notebook, must have the following permissions:

- `ADMIN_API_ACCESS`
- `CAN_ACCESS_USER_ACTIVITY`

## NOTE 1:

This notebook assumes usage from a local laptop or similar, using a `.env` file with the following format:

```bash
# Where to connect
DATAROBOT_ENDPOINT = 'https://datarobot.example.com'

# A valid API key from a user with the permissions above
DATAROBOT_API_TOKEN = ''
```

If you wish to use the the DataRobot notebook functionality, be sure to set the following environment variables:

- DATAROBOT_ENDPOINT
- DATAROBOT_API_TOKEN

And comment where shown in the notebook

## NOTE 2:
The block below uses pip3 to install the required python packages.
Be sure to update this command to what is appropreate for your notebook

# Python package import

In [ ]:
!pip3 install --upgrade pip
!pip3 install dotenv
!pip3 install datarobot
!pip3 install faker

# Initial Setup

In [ ]:
from datetime import datetime as dt
from datetime import timedelta
import json
import os
from random import randint
import urllib.parse
from urllib.parse import urlparse

from IPython.display import display, HTML
import datarobot as dr

########################################################
# Comment these 2 lines when used in DataRobot notebooks
from dotenv import find_dotenv, load_dotenv
from faker import Faker
import pandas as pd

_ = load_dotenv(find_dotenv())
########################################################

# Provide the URL protocol, address (IP or FQDN)
# Example: https://datarobot.example.com or http://10.1.2.3
DATAROBOT_ENDPOINT = os.environ["DATAROBOT_ENDPOINT"]

# Provide an API key from a user with the ADMIN_API_ACCESS permission from this cluster
DATAROBOT_API_TOKEN = os.environ["DATAROBOT_API_TOKEN"]

# These values are mostly constant
# Create the shared DataRobot client
my_client = dr.Client(
    token="%s" % (DATAROBOT_API_TOKEN),
    endpoint="%s/api/v2" % (DATAROBOT_ENDPOINT),
    ssl_verify=True if (urlparse(DATAROBOT_ENDPOINT)).scheme == "https" else False,
)

########################################################
# Define the download function


def findEventData(_client, location, parameters):
    """
    Uses the DR Client to get the data from the event logs
    """
    my_replies = []
    reply_set = json.loads(
        _client.get("%s?%s" % (location, urllib.parse.urlencode(parameters))).text
    )

    while "next" in reply_set and reply_set["next"] != None:
        for reply in reply_set["data"]:
            my_replies.append(reply)

        my_next = urlparse(reply_set["next"])
        reply_set = json.loads(_client.get("%s?%s" % (location, my_next.query)).text)

    # Add the last set of items, depending on where it lives
    for reply in reply_set["data"]:
        my_replies.append(reply)

    # print("getEventData %s item count:> %s" % ( location, len(my_replies)))
    return my_replies


########################################################
# Verbose settings statement
print("# -----------------------------------------")
print("DataRobot client version: %s" % dr.__version__)
print("Pandas version: %s" % pd.__version__)
print("# -----------------------------------------")
print("DATAROBOT_ENDPOINT: %s" % DATAROBOT_ENDPOINT)
print("DATAROBOT_API_TOKEN length: %s" % len(DATAROBOT_API_TOKEN))

# Load Everything and drop an entire column

If the query needs to be customized, see here for details: https://docs.datarobot.com/en/docs/api/reference/public-api/analytics.html#get-apiv2eventlogs

A complete example of the UAM query is below

```python
my_params = {
    'projectId': '<The project to select log records for>',
    'userId': '<The user to select log records for>',
    'orgId': '<The organization to select log records for>',
    'event' : '<The event type of records>',
    'minTimestamp': '<The lower bound for timestamps. E.g. 2016-12-13T11:12:13.141516Z>',
    'maxTimestamp': 'The upper bound for timestamps. E.g. 2024-01-26T16:10:42.234516Z',
    'offset': '<This many results will be skipped. Defaults to 0>'
    'order': '<The order of the results. Defaults to descending>'
}
```

In [ ]:
# The Example below show for the last 30 days
end_date = dt.utcnow()
start_date = end_date - timedelta(days=30)

my_params = {"minTimestamp": start_date, "maxTimestamp": end_date}

# # This is where you can update the Event Log search params
# # For now, we are extracting the default events
# my_params = {}

print("Loading data from: %s/eventLogs" % my_client.endpoint)
print("params: %s" % my_params)

EVENTS = findEventData(my_client, "eventLogs", my_params)
print("Currnet log entries: %s" % len(EVENTS))

MASTER_EVENTS_DF = pd.json_normalize(EVENTS)

# Puts the scrollbar next to the DataFrame
display(
    HTML(
        "<div style='height: 600px; overflow: auto; width: fit-content'>"
        + MASTER_EVENTS_DF.style.to_html()
        + "</div>"
    )
)

## Part 2

In [ ]:
# To Drop an entire column:
REDUCED_DF = MASTER_EVENTS_DF
REDUCED_DF.drop(
    columns=["username"], inplace=True
)  # dropped because unique for every row

# Puts the scrollbar next to the DataFrame
display(
    HTML(
        "<div style='height: 600px; overflow: auto; width: fit-content'>"
        + REDUCED_DF.style.to_html()
        + "</div>"
    )
)

# Download a subset and consistently change a column

In [ ]:
CHANGED_DF = pd.json_normalize(EVENTS)
print("Got Data: %s" % len(CHANGED_DF.index))

# seed the random generator to produce the same results
faker = Faker()
Faker.seed()

# In this case we are replacing the username with the new
dict_names = {name: faker.name() for name in CHANGED_DF["username"].unique()}

# Handy debug statement
print("DEBUG dict_names: %s" % json.dumps(dict_names, indent=4))

CHANGED_DF["username"] = CHANGED_DF["username"].map(dict_names)

# Puts the scrollbar next to the DataFrame
display(
    HTML(
        "<div style='height: 600px; overflow: auto; width: fit-content'>"
        + CHANGED_DF.style.to_html()
        + "</div>"
    )
)

In [ ]:
# Save the file
CHANGED_DF.to_csv("./my_outputfile.csv", encoding="utf-8")

### Copyright 2024 DataRobot Inc

**All Rights Reserved.**

This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, express or implied.